# Cluster and HII-regions Single <a class="tocSkip">

the aim of this notebook is to combine the HII-region and cluster catalogues. This notebook is for use with a single galaxy at a time.

In [ ]:
# reload modules after they have been modified
%load_ext autoreload
%autoreload 2

from pnlf.packages import *

from pnlf.constants import tab10, single_column, two_column
from cluster.plot import quick_plot, add_scale

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
logging.basicConfig(stream=sys.stdout,datefmt='%H:%M:%S',level=logging.INFO)
logger = logging.getLogger(__name__)

basedir = Path('..')  # where we save stuff (and )
data_ext = Path('a:') # raw data

# we use the sample table for basic galaxy properties
sample_table = ascii.read(basedir/'..'/'pnlf'/'data'/'interim'/'sample.txt')
sample_table.add_index('name')
sample_table['SkyCoord'] = SkyCoord(sample_table['R.A.'],sample_table['Dec.'])

## Read in data

the galaxies listed in `hst_sample` have a cluster catalogue. The galaxies listed in `muse_sample` have astrosat observations to measure the FUV.

In [ ]:
hst_sample      = set(['NGC0628','NGC1365', 'NGC1433', 'NGC1566', 'NGC3351', 'NGC3627', 'NGC4535'])
astrosat_sample = set([x.stem.split('_')[0] for x in (data_ext/'Astrosat').iterdir() if x.is_file() and x.suffix=='.fits'])
muse_sample     = set(sample_table['name'])
complete_sample = hst_sample & astrosat_sample & muse_sample

name = 'NGC1365'

### MUSE (DAP + nebulae catalogues)

In [ ]:
from pnlf.auxiliary import filter_table
from pnlf.io import ReadLineMaps

p = {x:sample_table.loc[name][x] for x in sample_table.columns}

# DAP linemaps (Halpha and OIII)
filename = data_ext / 'MUSE_DR2.1' / 'MUSEDAP' / f'{name}_MAPS.fits'
with fits.open(filename) as hdul:
    Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                    uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                    mask=np.isnan(hdul['HA6562_FLUX'].data),
                    meta=hdul['HA6562_FLUX'].header,
                    wcs=WCS(hdul['HA6562_FLUX'].header))
    OIII = NDData(data=hdul['OIII5006_FLUX'].data,
                    uncertainty=StdDevUncertainty(hdul['OIII5006_FLUX_ERR'].data),
                    mask=np.isnan(hdul['OIII5006_FLUX'].data),
                    meta=hdul['OIII5006_FLUX'].header,
                    wcs=WCS(hdul['OIII5006_FLUX'].header)) 

path = data_ext / 'MUSE_DR2.1' / 'filterImages' 
sdss_g, h = fits.getdata(path / f'{name}_IMAGE_FOV_SDSS_g_WCS_Pall_mad.fits',header=True)
sdss_r, h = fits.getdata(path / f'{name}_IMAGE_FOV_SDSS_r_WCS_Pall_mad.fits',header=True)
sdss_i, h = fits.getdata(path / f'{name}_IMAGE_FOV_SDSS_i_WCS_Pall_mad.fits',header=True)
    
# nebulae catalogue from Francesco (mostly HII-regions)
filename = basedir / 'data' / 'interim' / 'Nebulae_Catalogue_with_FUV_eq_v2p1.fits'
with fits.open(filename) as hdul:
    nebulae = Table(hdul[1].data)
nebulae['SkyCoord'] = SkyCoord(nebulae['cen_ra']*u.deg,nebulae['cen_dec']*u.deg)
nebulae.rename_columns(['cen_x','cen_y'],['x','y'])
nebulae.add_index('region_ID')

with np.errstate(divide='ignore'):
    nebulae['[SIII]/[SII]'] = np.nan
    SII = nebulae['SII6716_FLUX_CORR']+nebulae['SII6730_FLUX_CORR']
    SIII = nebulae['SIII6312_FLUX_CORR']+nebulae['SIII9068_FLUX_CORR']
    nebulae[SII>0]['[SIII]/[SII]'] = SIII[SII>0]/SII[SII>0]
    nebulae['HA/FUV'] = nebulae['HA6562_FLUX_CORR']/nebulae['FUV_FLUX_CORR']
    nebulae['HA/FUV_err'] = nebulae['HA/FUV']*np.sqrt((nebulae['HA6562_FLUX_CORR_ERR']/nebulae['HA6562_FLUX_CORR'])**2+(nebulae['FUV_FLUX_CORR_ERR']/nebulae['FUV_FLUX_CORR'])**2)

nebulae['HIIregion'] = (nebulae['BPT_NII']==0) & (nebulae['BPT_SII']==0) & (nebulae['BPT_OI']==0)
HII_regions = filter_table(nebulae,gal_name=name,BPT_NII=0,BPT_SII=0,BPT_OI=0)
nebulae = filter_table(nebulae,gal_name=name)

filename = data_ext / 'MUSE_DR2.1' / 'Nebulae catalogue' /'spatial_masks'/f'{name}_nebulae_mask.fits'
with fits.open(filename) as hdul:
    nebulae_mask = NDData(hdul[0].data.astype(float),mask=Halpha.mask,meta=hdul[0].header,wcs=WCS(hdul[0].header))
    nebulae_mask.data[nebulae_mask.data==-1] = np.nan

# WFI image (larger FOV)
filename = data_ext / 'WFI' / f'{name}_Rc_flux_nosky.fits'
with fits.open(filename) as hdul:
    WFI = NDData(data=hdul[0].data,
                 meta=hdul[0].header,
                 wcs=WCS(hdul[0].header))
    
# most of the time we do not need the datacubes
if False:
    #from spectral_cube import SpectralCube
    filename = Path('g:') /'Archive'/'MUSE'/'DR2.1'/'datacubes'/f'{name}_DATACUBE_FINAL_WCS_Pall_mad.fits'
    with fits.open(filename , memmap=True, mode='denywrite') as hdul:
        data_cube   = hdul[1].data
        cube_header = hdul[1].header   
    
print(f'{name}: {len(HII_regions)} HII-regions in final catalogue')

### HST

**white light + filter images**

In [ ]:
from cluster.io import read_associations

target  = name.lower()
scalepc = 32

# whitelight image (we set 0s to nan)
with fits.open(data_ext / 'HST' / 'white_light' / f'{name.lower()}_white_24rgb.fits') as hdul:
    hst_whitelight = NDData(hdul[0].data,mask=hdul[0].data==0,meta=hdul[0].header,wcs=WCS(hdul[0].header))
    hst_whitelight.data[hst_whitelight.data==0] = np.nan
    
# filter image with uncertainties
filename = data_ext / 'HST' / 'filterImages' / f'{name.lower()}_uvis_f275w_exp_drc_sci.fits'
with fits.open(filename) as hdul:
    F275 = NDData(hdul[0].data,
                  mask=hdul[0].data==0,
                  meta=hdul[0].header,
                  wcs=WCS(hdul[0].header))
filename = data_ext / 'HST' / 'filterImages' / f'{name.lower()}_uvis_f275w_err_drc_sci.fits'
with fits.open(filename) as hdul:
    F275.uncertainty = StdDevUncertainty(hdul[0].data)
    
associations, associations_mask = read_associations(folder=data_ext/'HST',target=target,scalepc=scalepc)

# modify table (rename the columns such that the clusters and associations are identical)
associations['SkyCoord'] = SkyCoord(associations['reg_ra']*u.degree,associations['reg_dec']*u.degree)
associations.rename_columns(['reg_id','reg_ra','reg_dec','reg_x','reg_y',
                             'reg_dolflux_Age_MinChiSq','reg_dolflux_Mass_MinChiSq','reg_dolflux_Ebv_MinChiSq',
                             'reg_dolflux_Age_MinChiSq_err','reg_dolflux_Mass_MinChiSq_err','reg_dolflux_Ebv_MinChiSq_err'],
                            ['assoc_ID','RA','DEC','X','Y','age','mass','EBV','age_err','mass_err','EBV_err'])
for col in list(associations.columns):
    if col.endswith('mjy'):
        associations[f'{col.split("_")[0]}_FLUX'] = 1e20*associations[col]*u.mJy.to(u.erg/u.s/u.cm**2/u.Hz)
    if col.endswith('mjy_err'):
        associations[f'{col.split("_")[0]}_FLUX_ERR'] = 1e20*associations[col]*u.mJy.to(u.erg/u.s/u.cm**2/u.Hz)

print(f'{name}: {len(associations)} associations in catalogue')    
# associations mask

### Astrosat

https://uvit.iiap.res.in/Instrument/Filters

the resolution is 0.4" per pixel. With a PSF resolution of 1.8" this leads to fwhm ~ 4.5 px. This corresponds to a std = 1.91 px

In [ ]:
# whitelight image
astro_file = data_ext / 'Astrosat' / f'{name}_FUV_F148W_flux_reproj.fits'

if not astro_file.is_file():
    astro_file = data_ext / 'Astrosat' / f'{name}_FUV_F154W_flux_reproj.fits'
    if not astro_file.is_file():
        print(f'no astrosat file for {name}')
    
with fits.open(astro_file) as hdul:
    astrosat = NDData(hdul[0].data,meta=hdul[0].header,wcs=WCS(hdul[0].header))

## Equivalent Width

the first step is to extract the spectra of each HII-region.

Are the spectra continuum subtracted?

In [ ]:
from astropy.visualization import quantity_support
quantity_support()

filename = data_ext / 'MUSE_DR2.1' / 'Nebulae catalogue' /'spectra'/f'{name}_VorSpectra.fits'
with fits.open(filename) as hdul:
    spectra = Table(hdul[1].data)
    spectral_axis = np.exp(Table(hdul[2].data)['LOGLAM'])*u.Angstrom
    
spectra['region_ID'] = np.arange(len(spectra))
spectra.add_index('region_ID')

In [ ]:
H0 = 67 * u.km / u.s / u.Mpc
z = (H0*Distance(distmod=p['(m-M)'])/c.c).decompose()
lam_HA0 = 6562.8*u.Angstrom
lam_HA = (1+z)*lam_HA0

lam_HA

In [ ]:
from cluster.spectrum import fit_emission_line

region_ID = 10
#lam_HA = 6595*u.Angstrom
filename = basedir/'reports'/name/f'{name}_eqwidth.png'
flux = spectra.loc[region_ID]['SPEC']*u.erg/u.s/u.cm**2/u.A
fit = fit_emission_line(spectral_axis,flux,lam_HA,filename=filename)
integrated_flux = fit.amplitude_0*np.sqrt(np.pi)*np.exp(-1/(2*fit.stddev_0**2)) * u.erg/u.s/u.cm**2

print(f"f_cat/f_fit = {nebulae.loc[region_ID]['HA6562_FLUX']/integrated_flux.value:.2f}")

In [ ]:
from tqdm import tqdm

#lam_HA = 6595*u.Angstrom

HA = []
HII_regions['eq_width'] = np.nan
for region_ID in tqdm(HII_regions['region_ID']):
    
    flux = spectra.loc[region_ID]['SPEC']*u.erg/u.s/u.cm**2/u.A
    fit = fit_emission_line(spectral_axis,flux,lam_HA,plot=False)
    integrated_flux = fit.amplitude_0*np.sqrt(np.pi)*np.exp(-1/(2*fit.stddev_0**2)) * u.erg/u.s/u.cm**2
    continuum = fit.c0_1 * u.erg/u.s/u.cm**2/u.Angstrom
    eq_width = integrated_flux/continuum
    eq_width = HII_regions.loc[region_ID]['HA6562_FLUX']/continuum
    HII_regions.loc[region_ID]['eq_width'] = eq_width.value
    
    HA.append(integrated_flux)
    #HA_cat = nebulae.loc[region_ID]['HA6562_FLUX']
    #print(f'{integrated_flux/HA_cat:.2f}')
    #print(f'HA = {fit.mean_0.value:.2f}')
HA = np.array([x.value for x in HA])

In [ ]:
fig,ax=plt.subplots(figsize=(5,5))

ax.scatter(HA,HII_regions['HA6562_FLUX']/2.5)
x = np.linspace(0,2e6)
ax.plot(x,x,color='black')
ax.set(ylim=[0,2e6],xlim=[0,2e6])
plt.show()

## HST and MUSE

### compare footprints of different observations

here we compare the footprints of the observations and check which objects overlap. The FOV of astrosat is a circle with a diameter of 28'. This is much larger than HST and MUSE and both will always be covered by the astrosat observations

In [ ]:
from cluster.regions import find_sky_region

reg_muse_pix, reg_muse_sky = find_sky_region(nebulae_mask.mask.astype(int),wcs=nebulae_mask.wcs)
reg_hst_pix, reg_hst_sky = find_sky_region(hst_whitelight.mask.astype(int),wcs=hst_whitelight.wcs)

# check which nebulae/clusters are within the HST/MUSE FOV
associations['in_frame'] = reg_muse_sky.contains(associations['SkyCoord'],nebulae_mask.wcs)
#clusters['in_frame'] = reg_muse_sky.contains(clusters['SkyCoord'],nebulae_mask.wcs)
nebulae['in_frame']  = reg_hst_sky.contains(nebulae['SkyCoord'],nebulae_mask.wcs)

print(f'{np.sum(associations["in_frame"])} (of {len(associations)}) associations in MUSE FOV')
#print(f'{np.sum(clusters["in_frame"])} (of {len(clusters)}) clusters in MUSE FOV')
print(f'{np.sum(nebulae["in_frame"])} (of {len(nebulae)}) nebulae in HST FOV')

In [ ]:
WFI_cutout = Cutout2D(WFI.data,p['SkyCoord'],size=6*u.arcmin,wcs=WFI.wcs)

# project from muse to hst coordinates
reg_muse_wfi = reg_muse_sky.to_pixel(WFI_cutout.wcs)
reg_hst_wfi  = reg_hst_sky.to_pixel(WFI_cutout.wcs)

# plot image
ax = quick_plot(WFI_cutout,figsize=(single_column,single_column),cmap=plt.cm.gray)
add_scale(ax,u.arcmin,label="1'",color='white',fontsize=10)

reg_muse_wfi.plot(ax=ax,ec='tab:red',label='MUSE')
reg_hst_wfi.plot(ax=ax,ec='tab:orange',label='HST')

#ax.set(xlim=[3000,11000],ylim=[3000,11000])
plt.savefig(basedir/'reports'/name/'footpring.pdf',dpi=600)
plt.show()

## Association and nebulae

the association catalogue differs from the clusters in that its entries are extended. Because we match two catalogues with extended objects, we must proceed differently.

In a first step we take a look at a single association

In [ ]:
from skimage.measure import find_contours
from regions import PixCoord, PolygonPixelRegion

assoc_ID = 10
pos = associations['SkyCoord'][associations['assoc_ID']==assoc_ID]

contours = find_contours(associations_mask.data==assoc_ID,0.5,)
coords = max(contours,key=len)

# the coordinates from find_counters are switched compared to astropy
reg_pix  = PolygonPixelRegion(vertices = PixCoord(*coords.T[::-1])) 
reg_sky  = reg_pix.to_sky(associations_mask.wcs)

mask_cutout = Cutout2D(associations_mask.data,pos,size=1*u.arcsecond,wcs=associations_mask.wcs)
F275_cutout = Cutout2D(F275.data,pos,size=2*u.arcsecond,wcs=F275.wcs)

reg_pix_cut  = reg_sky.to_pixel(mask_cutout.wcs)

ax = quick_plot(F275_cutout,figsize=(single_column,single_column),cmap=plt.cm.gray)
ax.imshow(mask_cutout.data,alpha=0.5)

reg_pix_cut.plot(ax=ax,ec='tab:red',label='MUSE')

plt.show()

to match the nebulae to the associations we first reproject the mask of the nebulae to the HST image. We then scale the association mask by the number of associations (assume we have 1432 objects, then 615 becomes 0.0615). This way we can add the two masks together and infer from the resulting unique values which clusters overlap with which associations

### Match catalogues

and make some plots that showcase the position/overlap

```
# NGC3627 is too large to reproject
#center = SkyCoord(sample_table.loc[name]['R.A.'],sample_table.loc[name]['Dec.'])
#cutout = Cutout2D(associations_mask.data,center,size=(5.5*u.arcmin,3*u.arcmin),wcs=associations_mask.wcs)
cutout = Cutout2D(associations_mask.data,(7000,7000),size=(9000,7000),wcs=associations_mask.wcs)

nebulae_hst, _  = reproject_interp(nebulae_mask,
                                   output_projection=cutout.wcs,
                                   shape_out=cutout.data.shape,
                                   order='nearest-neighbor')    
scale = 10**np.ceil(np.log10(max(cutout.data[~np.isnan(cutout.data)])))
s_arr = cutout.data/scale+nebulae_hst
```

In [ ]:
from reproject import reproject_interp
# reproject nebulae mask to hst 
nebulae_hst, _  = reproject_interp(nebulae_mask,
                                   output_projection=associations_mask.wcs,
                                   shape_out=associations_mask.data.shape,
                                   order='nearest-neighbor')    

# we scale the associations such that the the id is in the decimal
scale = 10**np.ceil(np.log10(max(associations_mask.data[~np.isnan(associations_mask.data)])))
s_arr = associations_mask.data/scale+nebulae_hst

header = associations_mask.wcs.to_header()
header['scale'] = scale
hdu = fits.PrimaryHDU(s_arr,header=header)
hdu.writeto(basedir/'data'/'map_nebulae_association'/f'{name}_map.fits',overwrite=True)

In [ ]:
# ids of associations, nebulae and combination (sum) of both
a_id = np.unique(associations_mask.data[~np.isnan(associations_mask.data)]).astype(int)
n_id = np.unique(nebulae_hst[~np.isnan(nebulae_hst)]).astype(int)
s_id = np.unique(s_arr[~np.isnan(s_arr)])

# this splits the sum into two parts (nebulae and associations)
a_modf,n_modf = np.modf(s_id)
n_modf = n_modf.astype(int)
a_modf = np.round(a_modf*scale).astype(int)

unique_a, count_a = np.unique(a_modf,return_counts=True)
unique_n, count_n = np.unique(n_modf,return_counts=True)

nebulae_dict = {int(n) : a_modf[n_modf==n].tolist() for n in n_id}     
associations_dict = {int(a) : n_modf[a_modf==a].tolist() for a in a_id}     


# so far we ensured that the nebulae in unique_n have only one association,
# but it is possible that this association goes beyond the nebulae and into
# a second nebulae. Those objects are excluded here
isolated_nebulae = set()
isolated_assoc   = set()
for n,v in nebulae_dict.items():
    if len(v)==1:
        if len(associations_dict[v[0]])==1:
            isolated_nebulae.add(n)
            isolated_assoc.add(v[0])
            
print(f'n_associations = {len(associations_dict)}')
print(f'n_nebulae      = {len(nebulae_dict)}')
print(f'1to1 match     = {len(isolated_nebulae)}')


# we save those two dicts so we do not have to redo this everytime
with open(basedir/'data'/'map_nebulae_association'/f'{name}_nebulae.yml','w+') as f:
    yaml.dump(nebulae_dict,f)
with open(basedir/'data'/'map_nebulae_association'/f'{name}_associations.yml','w+') as f:
    yaml.dump(associations_dict,f)

#### for associations

In [ ]:
# find all assoc that have at least one pixel outside of the nebulae masks
mask = associations_mask.data.copy()
mask[~np.isnan(nebulae_hst)] = np.nan
outside = np.unique(mask[~np.isnan(mask)].astype(int))

# find all assoc that have at least one pixel inside of the nebulea masks
mask = associations_mask.data.copy()
mask[np.isnan(nebulae_hst)] = np.nan
inside = np.unique(mask[~np.isnan(mask)].astype(int))

contained = np.setdiff1d(inside,outside)
partial   = np.intersect1d(inside,outside)
isolated  = np.setdiff1d(outside,inside)

print(f'contained: {len(contained)}\npartial: {len(partial)}\nisolated: {len(isolated)}')

In [ ]:
assoc_tmp = associations[['assoc_ID']].copy()
assoc_tmp.add_index('assoc_ID')

assoc_tmp['overlap'] = np.empty(len(associations),dtype='U9')
assoc_tmp['overlap'][np.isin(assoc_tmp['assoc_ID'],contained)] = 'contained'
assoc_tmp['overlap'][np.isin(assoc_tmp['assoc_ID'],partial)]   = 'partial'
assoc_tmp['overlap'][np.isin(assoc_tmp['assoc_ID'],isolated)]  = 'isolated'
assoc_tmp['1to1'] = False
assoc_tmp['1to1'][np.isin(assoc_tmp['assoc_ID'],list(isolated_assoc))] = True
assoc_tmp['Nnebulae'] = [len(associations_dict[k]) for k in assoc_tmp['assoc_ID']]

assoc_tmp['region_ID'] = np.nan
assoc_tmp['region_ID'][assoc_tmp['1to1']] = [associations_dict[k][0] for k in assoc_tmp[assoc_tmp['1to1']]['assoc_ID']]

hdu = fits.BinTableHDU(assoc_tmp,name='joined catalogue')
hdu.writeto(basedir/'data'/'map_nebulae_association'/f'{name}_associations.fits',overwrite=True)

#### for nebulae

In [ ]:
from cluster.regions import find_neighbors
from tqdm import tqdm 

nebulae_tmp = nebulae[['region_ID','x','y']].copy()
nebulae_tmp.add_index('region_ID')

nebulae_tmp['neighbors'] = np.nan
for row in tqdm(nebulae_tmp):
    row['neighbors'] = len(find_neighbors(nebulae_mask.data,tuple(row[['x','y']]),row['region_ID'],plot=False))
del nebulae_tmp[['x','y']]

nebulae_tmp['1to1'] = False
nebulae_tmp['1to1'][np.isin(nebulae_tmp['region_ID'],list(isolated_nebulae))] = True
nebulae_tmp['Nassoc'] = [len(nebulae_dict[k]) for k in nebulae_tmp['region_ID']]
nebulae_tmp['assoc_ID'] = np.nan
nebulae_tmp['assoc_ID'][nebulae_tmp['1to1']] = [nebulae_dict[k][0] for k in nebulae_tmp[nebulae_tmp['1to1']]['region_ID']]
    
hdu = fits.BinTableHDU(nebulae_tmp,name='joined catalogue')
hdu.writeto(basedir/'data'/'map_nebulae_association'/f'{name}_nebulae.fits',overwrite=True)
#del nebulae_tmp['1to1']

print(f'{np.sum(nebulae_tmp["neighbors"]==0)} nebulae have no neighbors')

#### join catalogues

In [ ]:
from astropy.table import join
from astropy.coordinates import match_coordinates_sky

catalogue = join(assoc_tmp,nebulae_tmp,keys=['assoc_ID','region_ID'])
catalogue = join(catalogue,nebulae,keys='region_ID')
catalogue = join(catalogue,associations,keys='assoc_ID')

catalogue.rename_columns(['X','Y','x','y','RA','DEC','cen_ra','cen_dec','reg_area','region_area',
                          'EBV_1','EBV_2','EBV_err','EBV_ERR','SkyCoord_1','SkyCoord_2'],
                         ['x_asc','y_asc','x_neb','y_neb','ra_asc','dec_asc','ra_neb','dec_neb',
                          'area_asc','area_neb','EBV_stars','EBV_balmer','EBV_stars_err','EBV_balmer_err',
                          'SkyCoord_asc','SkyCoord_neb'])

# separation to other associations and nebulae
idx,sep_asc,_= match_coordinates_sky(catalogue['SkyCoord_asc'],associations['SkyCoord'],nthneighbor=2)
idx,sep_neb,_= match_coordinates_sky(catalogue['SkyCoord_neb'],nebulae['SkyCoord'],nthneighbor=2)
catalogue['sep_asc'] = sep_asc.to(u.arcsec)
catalogue['sep_neb'] = sep_neb.to(u.arcsec)

# select the columns of the joined catalogue
columns = ['assoc_ID','region_ID','x_asc','y_asc','x_neb','y_neb',
           'ra_asc','dec_asc','ra_neb','dec_neb','SkyCoord_asc','SkyCoord_neb','area_asc','area_neb',
           'sep_asc','sep_neb','neighbors','overlap',
           'age','age_err','mass','mass_err','EBV_stars','EBV_stars_err','EBV_balmer','EBV_balmer_err',
           'met_scal','met_scal_err','logq_D91','logq_D91_err',] + \
            [x for x in HII_regions.columns if x.endswith('_FLUX_CORR')] + \
            [x for x in HII_regions.columns if x.endswith('_FLUX_CORR_ERR')] + \
            ['NUV_FLUX','NUV_FLUX_ERR','U_FLUX','U_FLUX_ERR','B_FLUX','B_FLUX_ERR',
             'V_FLUX','V_FLUX_ERR','I_FLUX','I_FLUX_ERR'] + \
            ['HA/FUV','eq_width']
catalogue = catalogue[columns]
        
catalogue.rename_columns([col for col in catalogue.columns if col.endswith('FLUX_CORR')],
                      [col.replace('FLUX_CORR','flux') for col in catalogue.columns if col.endswith('FLUX_CORR')])
catalogue.rename_columns([col for col in catalogue.columns if col.endswith('FLUX_CORR_ERR')],
                      [col.replace('FLUX_CORR_ERR','flux_err') for col in catalogue.columns if col.endswith('FLUX_CORR_ERR')])
catalogue['assoc_ID'] = catalogue['assoc_ID'].astype('int')
catalogue['region_ID'] = catalogue['region_ID'].astype('int')

catalogue.info.description = 'Joined catalogue between associations and nebulae'
mean_sep = np.mean(catalogue['SkyCoord_asc'].separation(catalogue['SkyCoord_neb']))
print(f'{len(catalogue)} objects in catalogue')
print(f'the mean separation between cluster and association center is {mean_sep.to(u.arcsecond):.2f}')

#### Write
export parts of the joined catalogue (right now only the fully contained objects)

In [ ]:
export = catalogue.copy() #[catalogue['contained']]
#export.add_column(export['SkyCoord_asc'].to_string(style='hmsdms',precision=2),index=6,name='RaDec_asc')
#export.add_column(export['SkyCoord_neb'].to_string(style='hmsdms',precision=2),index=8,name='RaDec_neb')

RA_asc ,DEC_asc = zip(*[x.split(' ') for x in export['SkyCoord_asc'].to_string(style='hmsdms',precision=2)])
RA_neb ,DEC_neb = zip(*[x.split(' ') for x in export['SkyCoord_neb'].to_string(style='hmsdms',precision=2)])

export.add_column(RA_asc,index=6,name='Ra_asc')
export.add_column(DEC_asc,index=8,name='Dec_asc')
export.add_column(RA_neb,index=10,name='Ra_neb')
export.add_column(DEC_neb,index=12,name='Dec_neb')

for col in export.columns:
    if col not in ['Ra_asc','Dec_asc','Ra_neb','Dec_neb','region_ID','cluster_ID','overlap']:
        export[col].info.format = '%.2f'

del export[['ra_asc','dec_asc','ra_neb','dec_neb','SkyCoord_neb','SkyCoord_asc','HA/FUV']]

hdu = fits.BinTableHDU(export,name='joined catalogue')
hdu.writeto(basedir/'data'/'map_nebulae_association'/f'{name}_associations_and_nebulae_joined.fits',overwrite=True)

#### Read

In [ ]:
with open(basedir/'data'/'map_nebulae_association'/f'{name}_nebulae.yml') as f:
    nebulae_dict = yaml.load(f,Loader=yaml.SafeLoader)
with open(basedir/'data'/'map_nebulae_association'/f'{name}_associations.yml') as f:
    associations_dict = yaml.load(f,Loader=yaml.SafeLoader)
    
filename = basedir/'data'/'map_nebulae_association'/f'{name}_associations.fits'
assoc_tmp = Table(fits.getdata(filename,ext=1))
associations=join(associations,assoc_tmp,keys='assoc_ID')

filename = basedir/'data'/'map_nebulae_association'/f'{name}_nebulae.fits'
nebulae_tmp = Table(fits.getdata(filename,ext=1))
nebulae=join(nebulae,nebulae_tmp,keys='region_ID')

# read in existing catalogues
filename = basedir/'data'/'map_nebulae_association'/f'{name}_associations_and_nebulae_joined.fits'
catalogue = Table(fits.getdata(filename,ext=1))

catalogue['SkyCoord_asc'] = SkyCoord(catalogue['Ra_asc'],catalogue['Dec_asc'])
catalogue['SkyCoord_neb'] = SkyCoord(catalogue['Ra_neb'],catalogue['Dec_neb'])
catalogue['HA/FUV'] = catalogue['HA6562_flux']/catalogue['FUV_flux']
catalogue['HA/FUV_err'] = catalogue['HA/FUV']*np.sqrt((catalogue['HA6562_flux_err']/catalogue['HA6562_flux'])**2+(catalogue['FUV_flux_err']/catalogue['FUV_flux'])**2)

#### save cutout for each region in seperate fits file

In [ ]:
from reproject import reproject_interp 

def save_cutouts(position,size=4*u.arcsec):
    
    header = fits.Header()
    header['gal_name'] = name
    header['RA'] = row['SkyCoord_asc'].ra.to(u.degree).value
    header['DEC'] = row['SkyCoord_asc'].dec.to(u.degree).value
    header['RADESYS'] = 'ICRS'
    header['regionID'] = row['region_ID']
    header['assocID'] = row['cluster_ID']

    hdul = fits.HDUList([fits.PrimaryHDU(header=header)])

    # save HST image
    cutout = Cutout2D(F275.data,position=position,size=size,wcs=F275.wcs)
    hdul.append(fits.ImageHDU(cutout.data,header=cutout.wcs.to_header(),name='F275'))

    # save Halpha
    Halpha_cutout, _  = reproject_interp(Halpha,output_projection=cutout.wcs,shape_out=cutout.shape,order='bilinear')    
    hdul.append(fits.ImageHDU(Halpha_cutout,header=cutout.wcs.to_header(),name='Halpha'))


    # save OIII
    OIII_cutout, _  = reproject_interp(OIII,output_projection=cutout.wcs,shape_out=cutout.shape,order='bilinear')    
    hdul.append(fits.ImageHDU(OIII_cutout,header=cutout.wcs.to_header(),name='OIII'))


    # save nebulae mask
    nebulae_cutout, _  = reproject_interp(nebulae_mask,output_projection=cutout.wcs,shape_out=cutout.shape,order='nearest-neighbor')    
    hdul.append(fits.ImageHDU(nebulae_cutout,header=cutout.wcs.to_header(),name='nebulae'))


    # save association mask
    assoc_cutout, _  = reproject_interp(associations_mask,output_projection=cutout.wcs,shape_out=cutout.shape,order='nearest-neighbor')    
    hdul.append(fits.ImageHDU(assoc_cutout,header=cutout.wcs.to_header(),name='assoc'))

    hdul.writeto(basedir/'data'/'cutouts'/f'{name}_region{row["region_ID"]}.fits',overwrite=True,checksum=True)
    
    
row = catalogue[0]
position = row['SkyCoord_neb']

save_cutouts(position)

### Plot cutouts

look at the joined catalogue (containing only nebulae and clusters with a 1 to 1 relation)

In [ ]:
from cluster.plot import single_cutout

region_ID = 23
position = nebulae['SkyCoord'][nebulae['region_ID']==region_ID]

fig,ax=plt.subplots(figsize=(4,4))
single_cutout(ax=ax,position = position,
             image = F275,
             mask1 = nebulae_mask,
             mask2 = associations_mask,
             #points= clusters,
             #label= f'{region_ID}/{nebulae_dict[region_ID][0]}',
             size = 4*u.arcsecond)

#plt.savefig(basedir/'reports'/name/'single_region.png',dpi=315)
plt.show()

In [ ]:
from astropy.coordinates import match_coordinates_sky

offset = catalogue['SkyCoord_asc'].separation(catalogue['SkyCoord_neb'])
idx,sep,_=match_coordinates_sky(catalogue['SkyCoord_asc'],catalogue['SkyCoord_asc'],nthneighbor=2)


sample = catalogue[(catalogue['contained']) & (catalogue['neighbors']==0) & (offset<Angle('0.2"')) & (sep>Angle('5"'))]
print(f'{len(sample)} objects in catalogue')

In [ ]:
from cluster.plot import multi_cutout

#sample = catalogue[:12]

filename = basedir/'reports'/name/f'{name}_isolated_associations_F275'
positions = sample['SkyCoord_neb']
labels = [f'{ri}/{ci}' for ri, ci in sample[['region_ID','cluster_ID']]]

multi_cutout(positions = positions,
             image = F275,
             mask1 = nebulae_mask,
             mask2 = associations_mask,
             #points= clusters,
             labels= labels,
             size = 4*u.arcsecond,
             filename=filename,
             ncols=4)

create a multi page pdf with all isolated objects

In [ ]:
from cluster.plot import multi_page_cutout

filename = basedir/'reports'/name/f'{name}_isolated_associations_F275'
positions = catalogue['SkyCoord_neb'][catalogue['overlap']=='contained']
labels = [f'{ri}/{ci}' for ri, ci in catalogue[['region_ID','assoc_ID']][catalogue['overlap']=='contained']]

multi_page_cutout(positions = positions[:59],
             image = F275,
             mask1 = nebulae_mask,
             mask2 = associations_mask,
             #points= clusters,
             labels= labels,
             size = 4*u.arcsecond,
             filename=filename,
             ncols=5,nrows=4)

In [ ]:
WFI_cutout = Cutout2D(WFI.data,p['SkyCoord'],size=4*u.arcmin,wcs=WFI.wcs)

# project from muse to hst coordinates
reg_muse_wfi = reg_muse_sky.to_pixel(WFI_cutout.wcs)
reg_hst_wfi  = reg_hst_sky.to_pixel(WFI_cutout.wcs)

# plot image
fig,ax=plt.subplots(figsize=(two_column,two_column),subplot_kw={'projection': WFI_cutout.wcs})
norm = simple_norm(WFI_cutout.data,clip=False,percent=96)
ax.imshow(WFI_cutout.data,norm=norm,origin='lower',cmap=plt.cm.Greys)
add_scale(ax,u.arcmin,label="1'",color='white',fontsize=10)

reg_muse_wfi.plot(ax=ax,ec='tab:red',label='MUSE')
reg_hst_wfi.plot(ax=ax,ec='tab:orange',label='HST')

x,y = catalogue['SkyCoord_neb'][catalogue['contained']].to_pixel(WFI_cutout.wcs)
ax.scatter(x,y,marker='s',facecolors='none',s=30,lw=1,color='tab:red')

ax.coords[0].set_ticks_visible(False)
ax.coords[1].set_ticks_visible(False)
ax.coords[0].set_ticklabel_visible(False)
ax.coords[1].set_ticklabel_visible(False)
#ax.set(xlim=[3000,11000],ylim=[3000,11000])
plt.savefig(basedir/'reports'/name/f'{name}_location_in_galaxy.pdf',dpi=600)
plt.show()

### Correlations in joined catalogue

first we create a mask to select a subset of objects (e.g. based on mass)

In [ ]:
from astropy.coordinates import match_coordinates_sky
from cluster.auxiliary import bin_stat

# separation to other associations
idx,sep_others,_= match_coordinates_sky(catalogue['SkyCoord_asc'],associations['SkyCoord'],nthneighbor=2)
idx,sep_int,_= match_coordinates_sky(catalogue['SkyCoord_asc'],catalogue['SkyCoord_neb'])

# size of the association compared to the HII-region
small_HII = (catalogue['area_neb']/0.039) / (catalogue['area_asc']/11.95) > 2

# distance to centre of galaxy
galactic_center = SkyCoord(ra=p['R.A.'],dec=p['Dec.'])
catalogue['galactic_radius'] = catalogue['SkyCoord_neb'].separation(galactic_center).to(u.arcmin)

# define the criteria which objects we use in the plot
criteria = (catalogue['mass']>5e3) & catalogue['contained'] #& (catalogue['age']>catalogue['age_err']) 

#& catalogue['contained'] #& (catalogue['galactic_radius']>1*u.arcmin)

print(f'{np.sum(criteria)} objects match the criteria')
tmp = catalogue[criteria] #catalogue[criteria]

HA/FUV vs cluster age

In [ ]:
xlim = [0.5,10.5]

fig,ax=plt.subplots(figsize=(6,4))
ax.errorbar(tmp['age'],tmp['HA/FUV'],fmt='o')
            #xerr=tmp['age_err'],yerr=tmp['HA/FUV_err'])

x,mean,std = bin_stat(tmp['age'],tmp['HA/FUV'],xlim)
ax.errorbar(x,mean,yerr=std,fmt='-',color='black',label='mass / Msun')
ax.set(xlabel='Age/Myr',ylabel='Halpha / FUV',xlim=xlim)
#ax.set_title(r'only clusters with $M>10^{5}M_\odot$')

plt.savefig(basedir/'reports'/name/f'{name}_HaFUV_over_age.pdf',dpi=600)
plt.show()

equivalent width vs cluster age

In [ ]:
xlim = [0.5,10.5]

fig,ax=plt.subplots(figsize=(6,4))
sc= ax.scatter(tmp['age'],tmp['eq_width']) #,c=tmp['mass'],vmin=1e5,vmax=3e5)
#fig.colorbar(sc,label='mass / Msun')

x,mean,std = bin_stat(tmp['age'],tmp['eq_width'],xlim)
ax.errorbar(x,mean,yerr=std,fmt='-',color='black',label='mass / Msun')
ax.set(xlabel='Age/Myr',ylabel='equivalent width / Angstrom',xlim=xlim)
#ax.set_title(r'only clusters with $M>10^{5}M_\odot$')

plt.savefig(basedir/'reports'/name/f'{name}_eq_width_over_age.pdf',dpi=600)
plt.show()

In [ ]:
xlim = [0,100]
criteria = HII_regions['HA/FUV']<150

fig,ax=plt.subplots(figsize=(6,4))
sc= ax.scatter(HII_regions['eq_width'][criteria],HII_regions['HA/FUV'][criteria])

x,mean,std = bin_stat(HII_regions['eq_width'][criteria],HII_regions['HA/FUV'][criteria],xlim)
ax.errorbar(x,mean,yerr=std,fmt='-',color='black')
ax.set(xlabel='equivalent width / Angstrom',ylabel='Halpha / FUV',xlim=xlim,ylim=(0,70))
#ax.set_title(r'only clusters with $M>10^{5}M_\odot$')
plt.savefig(basedir/'reports'/name/f'{name}_HaFUV_over_eq_width.pdf',dpi=600)
plt.show()

Extinction from stars and from nebulae

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

# we expect EBV_balmer = 2 EBV_stars

fig = plt.figure(figsize=(single_column,single_column/1.1))
ax = fig.add_subplot()
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size="10%", pad=0.2,)

xlim = [0,10]
sc=ax.scatter(tmp['EBV_stars'],tmp['EBV_balmer'],c=tmp['age'],s=3,vmin=0,vmax=10,cmap=plt.cm.viridis_r)
ax.plot([0,1],[0,2],color='black')
ax.plot([0,2],[0,2],color='black')
fig.colorbar(sc,label='age / Myr',cax=cax)
ax.set(xlim=[0,0.7],ylim=[0,0.7],xlabel='E(B-V) stars',ylabel='E(B-V) Balmer')

plt.savefig(basedir/'reports'/name/f'{name}_EBV_Balmer_vs_Stars.pdf',dpi=600)
plt.show()

### Corner Plot

are there any obvious differences between eg the old, high EW regions versus young, low EW regions?

Likewise for the HA/FUV vs HA EQW plot, how much of this correlation is driven by Halpha, rather than a general trend?

In [ ]:
from cluster.plot import corner

catalogue['HA/NUV'] = catalogue['HA6562_flux'] / catalogue['NUV_FLUX'] / 1e12

filename = basedir/'reports'/name/f'{name}_corner'
columns  = ['age','HA/FUV','HA/NUV','met_scal','logq_D91']
limits   = {'age':(0,10),'eq_width':(0,100),'HA/FUV':(0,50),'HA/NUV':(0,20),'met_scal':(8.4,8.7),'logq_D91':(6,8)}

tmp = catalogue[catalogue['contained']]
print(f'sample contains {len(tmp)} objects')

corner(tmp,columns,limits,nbins=5,filename=filename,vmin=1000,vmax=1e6)

In [ ]:
from scipy.special import comb
from scipy.stats import spearmanr
import itertools

lines = [col for col in catalogue.columns if col.endswith('_flux')]
print(f'{len(lines)} different lines with {comb(len(lines),2)} possible combinations')

correlation = []
for pair in itertools.combinations(lines,2):
    not_nan = ~np.isnan(catalogue[pair[0]]) & ~np.isnan(catalogue[pair[1]])
    r,p = spearmanr(catalogue['age'],catalogue[pair[0]][not_nan]/catalogue[pair[1]][not_nan])
    correlation.append((r,pair))
a = [x for x in correlation if np.abs(x[0])>0.15]
a.sort(key=lambda x: np.abs(x[0]),reverse=True)
a

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))

rho,(line1,line2) = a[-1]
ax.scatter(catalogue['age'],catalogue[line1]/catalogue[line2])
ax.set(xlabel='age',ylabel=f"{line2.split('_')[0]}/{line1.split('_')[0]}")
plt.show()

In [ ]:
catalogue['HA/SII'] = catalogue['HA6562_flux'] / catalogue['SII6716_flux']
catalogue['HA/OI'] = catalogue['HA6562_flux'] / catalogue['OI6300_flux']
catalogue['HA/SII'][~np.isfinite(catalogue['HA/SII'])] = np.nan
catalogue['HA/OI'][~np.isfinite(catalogue['HA/OI'])] = np.nan

Halpha luminosity vs mass

In [ ]:
from cluster.auxiliary import bin_stat

xlim = [1e4,5e5]
print(f'{np.sum(criteria)} objects match the criteria')
tmp = catalogue[criteria] 

fig,ax=plt.subplots(figsize=(6,4))
ax.scatter(tmp['mass'],tmp['HA6562_flux'])
x,mean,std = bin_stat(tmp['mass'],tmp['HA6562_flux'],xlim)
#ax.errorbar(x,mean,yerr=std,fmt='-',color='black')
ax.set(xlabel='mass / Msun',ylabel='Halpha',
       xlim=xlim,ylim=[1e4,1e6],xscale='log',yscale='log')

#plt.savefig(basedir/'reports'/name/'HaFUV_over_age.pdf',dpi=600)
plt.show()

In [ ]:
from cluster.auxiliary import bin_stat

xlim = [1,2e5]
criteria = (catalogue['age']<20) #& (sep>Angle('3"'))
print(f'{np.sum(criteria)} objects match the criteria')
tmp = catalogue[criteria] #catalogue[criteria]

fig,ax=plt.subplots(figsize=(8,6))
ax.scatter(tmp['mass'],tmp['region_area'])
x,mean,std = bin_stat(tmp['mass'],tmp['region_area'],xlim)
ax.errorbar(x,mean,yerr=std,fmt='-',color='black')
ax.set(xlabel='mass / Msun',ylabel='HII-region area',xlim=xlim)

#plt.savefig(basedir/'reports'/name/'HaFUV_over_age.pdf',dpi=600)
plt.show()

### Associated vs isolated clusters

In [ ]:
fig,(ax1,ax2,ax3)=plt.subplots(ncols=3,figsize=(10,3))
bins = np.arange(0,10,1)

tmp = associations[(associations['mass']>1e3) & (associations['age']<10)]

ages_con = tmp[tmp['overlap']=='contained']['age']
ages_par = tmp[tmp['overlap']=='partial']['age']
ages_iso = tmp[tmp['overlap']=='isolated']['age']

print(f'ages: con={np.mean(ages_con):.2f}, par={np.mean(ages_par):.2f}, iso={np.mean(ages_iso):.2f}')

ax1.hist(ages_con,bins=bins,histtype='step',label='contained')
ax2.hist(ages_par,bins=bins,histtype='step',label='partially')
ax3.hist(ages_iso,bins=bins,histtype='step',label='isolated')

ax1.set_title(f'contained ({np.mean(ages_con):.2f} Myr)')
ax2.set_title(f'partially ({np.mean(ages_par):.2f} Myr)')
ax3.set_title(f'isolated ({np.mean(ages_iso):.2f} Myr)')

for ax in [ax1,ax2,ax3]:
    ax.set(ylim=[0,120],xlabel='age / Myr')
plt.savefig(basedir/'reports'/name/f'{name}_age_hist_contained.pdf',dpi=600)
plt.show()

In [ ]:
from astropy.coordinates import match_coordinates_sky

tmp = associations[(associations['mass']>1e3) & (associations['age']<10)]
idx,sep,_=match_coordinates_sky(tmp['SkyCoord'],nebulae['SkyCoord'])

ages1 = tmp[(sep<0.4*u.arcsec)]['age']
ages2 = tmp[(sep>0.4*u.arcsec) & (sep<0.8*u.arcsec)]['age']
ages3 = tmp[(sep>0.8*u.arcsec)]['age']

print(f'mean age: 1={np.mean(ages1):.2f}, 2={np.mean(ages2):.2f}, 3={np.mean(ages3):.2f} Myr')

fig,(ax1,ax2,ax3)=plt.subplots(ncols=3,figsize=(10,3))
bins = np.arange(0,10,1)

ax1.hist(ages1,bins=bins,histtype='step',label='isolated')
ax2.hist(ages2,bins=bins,histtype='step',label='partially')
ax3.hist(ages3,bins=bins,histtype='step',label='contained')
ax1.set_title(r'$s<0.4"$'+f' ({np.mean(ages1):.2f} Myr)')
ax2.set_title(r'$0.4"<s<0.8"$' +f' ({np.mean(ages2):.2f} Myr)')
ax3.set_title(r'$0.8"<s$'+f' ({np.mean(ages3):.2f} Myr)')

for ax in [ax1,ax2,ax3]:
    ax.set(ylim=[0,100],xlabel='age / Myr')
plt.savefig(basedir/'reports'/name/f'{name}_age_hist_sep.pdf',dpi=600)

plt.show()

## Measure FUV and Halpha at association position

### From point

If I select an aperture smaller than a pixel, the measured flux is directly proportional to the apertuer size. Therefore it doesn't matter that the astrosat resolution is much worse than HST or MUSE

In [ ]:
from dust_extinction.parameter_averages import O94, CCM89

extinction_model = CCM89(Rv=3.1)

def extinction(EBV,EBV_err,wavelength,plot=False):
    '''Calculate the extinction for a given EBV and wavelength with errors'''
    
    EBV = np.atleast_1d(EBV)
    sample_size = 100000

    ext = extinction_model.extinguish(wavelength,Ebv=EBV)
    
    EBV_rand = np.random.normal(loc=EBV,scale=EBV_err,size=(sample_size,len(EBV)))
    ext_arr  = extinction_model.extinguish(wavelength,Ebv=EBV_rand)
        
    ext_err  = np.std(ext_arr,axis=0)
    ext_mean = np.mean(ext_arr,axis=0)
    
    if plot:
        fig,(ax1,ax2) =plt.subplots(nrows=1,ncols=2,figsize=(two_column,two_column/2))
        ax1.hist(EBV_rand[:,0],bins=100)
        ax1.axvline(EBV[0],color='black')
        ax1.set(xlabel='E(B-V)')
        ax2.hist(ext_arr[:,0],bins=100)
        ax2.axvline(ext[0],color='black')
        ax2.set(xlabel='extinction')
        plt.show()
 
    return ext,ext_err

In [ ]:
from photutils import SkyCircularAperture,SkyCircularAnnulus,aperture_photometry

criteria = np.isin(associations['cluster_ID'],isolated_assoc)

aperture_size = 1*u.arcsecond
positions = associations['SkyCoord'][criteria]

aperture = SkyCircularAperture(positions,aperture_size)

fluxes = associations[['cluster_ID','SkyCoord','age','age_err','mass','mass_err','EBV','EBV_err']][criteria]
fluxes['FUV'] = 1e20*aperture_photometry(astrosat,aperture)['aperture_sum']
fluxes['HA'] = aperture_photometry(Halpha,aperture)['aperture_sum']



# because the HII-regions are sometimes extended and not circular, this is probably not sufficient
'''
r_in,r_out = 5*u.arcsec,8*u.arcsec
A_circle  = np.pi*aperture_size**2
A_annulus = np.pi*(r_out**2-r_in**2)
annulus_aperture = SkyCircularAnnulus(positions,r_in=r_in, r_out=r_out)


fluxes['FUV_bkg'] = 1e20*aperture_photometry(astrosat,annulus_aperture)['aperture_sum']/A_annulus*A_circle
fluxes['HA_bkg'] = aperture_photometry(Halpha,annulus_aperture)['aperture_sum']/A_annulus*A_circle
fluxes['FUV'] = fluxes['FUV']-fluxes['FUV_bkg']
fluxes['HA']  = fluxes['HA']-fluxes['HA_bkg']
'''

# E(B-V) is estimated from nebulae. E(B-V)_star = 0.5 E(B-V)_nebulae. FUV comes directly from stars
extinction_mw  = extinction_model.extinguish(1481*u.angstrom,Ebv=0.5*p['E(B-V)'])
ext_int,ext_int_err = extinction(associations['EBV'][criteria],associations['EBV_err'][criteria],wavelength=1481*u.angstrom)
fluxes['FUV'] = fluxes['FUV'] / extinction_mw 
fluxes['FUV_CORR'] = fluxes['FUV'] / ext_int 

# the Halpha line maps are already MW extinction corrected
ext_int,ext_int_err = extinction(2*associations['EBV'][criteria],associations['EBV_err'][criteria],wavelength=6562*u.angstrom)
fluxes['HA_CORR'] = fluxes['HA'] / ext_int 

In [ ]:
from cluster.auxiliary import bin_stat

bins = 10
xlim=[0.5,10.5]
fig,ax=plt.subplots(figsize=(6,6))

ax.scatter(fluxes['age'],fluxes['HA']/fluxes['FUV'])

x,mean,std = bin_stat(fluxes['age'],fluxes['HA']/fluxes['FUV'],xlim)
ax.errorbar(x,mean,yerr=std,fmt='-',color='black')
ax.set(xlabel='Age/Myr',ylabel='Halpha / FUV',xlim=xlim,ylim=[-10,125])

plt.show()

In [ ]:
from cluster.auxiliary import bin_stat

bins = 10
xlim=[0.5,10.5]
fig,ax=plt.subplots(figsize=(6,6))

ax.scatter(fluxes['age'],fluxes['HA_CORR']/fluxes['FUV_CORR'])

x,mean,std = bin_stat(fluxes['age'],fluxes['HA_CORR']/fluxes['FUV_CORR'],xlim)
ax.errorbar(x,mean,yerr=std,fmt='-',color='black')
ax.set(xlabel='Age/Myr',ylabel='Halpha / FUV',xlim=xlim,ylim=[-10,70])


plt.show()

### From mask

because the resolution of MUSE and astrosat is so much worse than HST, many associations won't be resolved and hence we can not measure the fluxes

In [ ]:
from reproject import reproject_interp

associations_muse, _  = reproject_interp(associations_mask,output_projection=Halpha.wcs,shape_out=Halpha.data.shape,order='nearest-neighbor') 
associations_astro, _ = reproject_interp(associations_mask,output_projection=astrosat.wcs,shape_out=astrosat.data.shape,order='nearest-neighbor')    

In [ ]:
sample = list(set(np.unique(associations_muse[~np.isnan(associations_muse)])) & set(np.unique(associations_astro[~np.isnan(associations_astro)])))
sample.sort()
HA_flux = [np.sum(Halpha.data[associations_muse==cluster_ID]) for cluster_ID in sample]
FUV_flux = [np.sum(astrosat.data[associations_astro==cluster_ID]) for cluster_ID in sample]

In [ ]:
from astropy.table import join

fluxes = Table([sample,HA_flux,FUV_flux],names=['cluster_ID','HA','FUV'])
catalogue = join(associations,fluxes[(~np.isnan(HA_flux)) & (~np.isnan(FUV_flux))],keys='cluster_ID')

In [ ]:
bins = 10
xlim=[0,100]
fig,ax=plt.subplots(figsize=(6,6))

ax.scatter(catalogue['age'],catalogue['HA']/catalogue['FUV'])

ax.set(xlim=xlim,xlabel='age/Myr',ylabel='Ha/FUV')
plt.show()

### problem with the resolution

In [ ]:
from reproject import reproject_interp, reproject_exact
from astropy.nddata import block_replicate

nebulae_astrosat, _ = reproject_interp(nebulae_mask,output_projection=astrosat.wcs,shape_out=astrosat.data.shape,order='nearest-neighbor') 
astro_MUSE, _ = reproject_exact(astrosat,output_projection=Halpha.wcs,shape_out=Halpha.data.shape)    
asttro_fine = block_replicate(astrosat,4)

In [ ]:
region_ID = 13
position = nebulae['SkyCoord'][nebulae['region_ID']==region_ID]
size = 4*u.arcsecond

nebulae_cutout       = Cutout2D(nebulae_mask.data,position,size=size,wcs=nebulae_mask.wcs)
astrosat_cutout_MUSE = Cutout2D(astro_MUSE,position,size=size,wcs=Halpha.wcs)

nebulae_cutout_astrosat = Cutout2D(nebulae_astrosat,position,size=size,wcs=astrosat.wcs)
astrosat_cutout = Cutout2D(astrosat.data,position,size=size,wcs=astrosat.wcs)

astrosat_up = block_replicate(astrosat_cutout.data,4)
nebulae_fine, _ = reproject_interp(nebulae_mask,output_projection=astrosat_cutout.wcs,shape_out=astrosat_up.shape,order='nearest-neighbor') 

In [ ]:
from skimage.measure import find_contours

fig,(ax1,ax2,ax3) = plt.subplots(ncols=3,figsize=(10,5))

neb_contours = []
for i in np.unique(nebulae_cutout.data[~np.isnan(nebulae_cutout.data)]):
    blank_mask = np.zeros_like(nebulae_cutout.data)
    blank_mask[nebulae_cutout.data==i] = 1
    neb_contours += find_contours(blank_mask, 0.5)
    
neb_contours_astrosat = []
for i in np.unique(nebulae_cutout_astrosat.data[~np.isnan(nebulae_cutout_astrosat.data)]):
    blank_mask = np.zeros_like(nebulae_cutout_astrosat.data)
    blank_mask[nebulae_cutout_astrosat.data==i] = 1
    neb_contours_astrosat += find_contours(blank_mask, 0.5)

neb_contours_fine = []
for i in np.unique(nebulae_fine[~np.isnan(nebulae_fine)]):
    blank_mask = np.zeros_like(nebulae_fine)
    blank_mask[nebulae_fine==i] = 1
    neb_contours_fine += find_contours(blank_mask, 0.5)
    
norm1 = simple_norm(astrosat_cutout.data,clip=False,percent=99)
ax1.imshow(astrosat_cutout.data,norm=norm1)

norm2 = simple_norm(astrosat_cutout_MUSE.data,clip=False,percent=99)
ax2.imshow(astrosat_cutout_MUSE.data,norm=norm2)

norm3 = simple_norm(astrosat_up,clip=False,percent=99)
ax3.imshow(astrosat_up,norm=norm3)

for coords in neb_contours:
    ax2.plot(coords[:,1],coords[:,0],color='tab:blue',lw=0.8)
for coords in neb_contours_astrosat:
    ax1.plot(coords[:,1],coords[:,0],color='tab:red',lw=0.8)   
for coords in neb_contours_fine:
    ax3.plot(coords[:,1],coords[:,0],color='tab:red',lw=0.8)  
    
ax1.set_title('original astrosat resolution')
ax2.set_title('interpolated to MUSE resolution')
    
plt.show()

In [ ]:
print(f'interpolate regions: {np.sum(astrosat_cutout.data[nebulae_cutout_astrosat.data==region_ID]):.2g}')
print(f'interpolate astrosat: {np.sum(np.sum(astrosat_cutout_MUSE.data[nebulae_cutout.data==region_ID])):.2g}')


In [ ]:
from reproject import reproject_exact

ast_small , _  = reproject_exact(astrosat,output_projection=neb_cutout.wcs,shape_out=(101,101)) 
ast_large , _  = reproject_exact(astrosat,output_projection=neb_cutout.wcs,shape_out=(202,202)) 
ast_org = Cutout2D(astrosat.data,position,size=size,wcs=astrosat.wcs)

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(ncols=3,figsize=(10,5))

ax1.imshow(ast_org.data)
ax2.imshow(ast_small)
ax3.imshow(ast_large)

plt.show()

In [ ]:

arr = np.array([[1,2],[3,4]])


astrosat_upsampled = block_replicate(astrosat,4,conserve_sum=True)


## Starburst99

compare our observations with simulated data

**Note**: the GENEVAHIGH 23 (Z=0.008) model used a metallicity of 0.02 for the high resolution models 

In [ ]:
from starburst import Cluster

cluster = Cluster(stellar_model='GENEVAv40',metallicity=0.014)

### Multiple Populations

In [ ]:
# test what multiple stellar populations would look like

dT = 0.5*np.array([1,2,3,4,5]) * u.Myr

time = cluster.ewidth['Time']
Ha  = cluster.ewidth['Luminosity_H_A'].copy()
FUV = cluster.FUV['FUV'].copy()

fig,(ax1,ax2,ax3) =plt.subplots(ncols=3,figsize=(1.5*two_column,two_column/2))

if False:
    ax1.plot(time.value/1e6,Ha,label=f'{0*u.Myr}')
    ax2.plot(time.value/1e6,FUV,label=f'{0*u.Myr}')   
    ax3.plot(time.value/1e6,Ha/FUV,label=f'{0*u.Myr}')   

for t in dT:
    c0 = cluster.time_shift(t)
    Ha_new  = np.interp(time,c0.ewidth['Time'],c0.ewidth['Luminosity_H_A'],left=0,right=0)
    FUV_new = np.interp(time,c0.FUV['Time'],c0.FUV['FUV'],left=0,right=0)   
     
    ax1.axvline(t.value,color='black',ls='--')
    ax2.axvline(t.value,color='black',ls='--')
    #ax3.axvline(t.value,color='black',ls='--')
    
    if False:
        ax1.plot(c0.ewidth['Time'].value/1e6,c0.ewidth['Luminosity_H_A'],label=f'{t}')
        ax2.plot(c0.FUV['Time'].value/1e6,c0.FUV['FUV'],label=f'{t}')   
        ax3.plot(c0.FUV['Time'].value/1e6,c0.ewidth['Luminosity_H_A']/c0.FUV['FUV'],label=f'{t}')    

    Ha += Ha_new
    FUV += FUV_new
    
ax1.plot(time.value/1e6,Ha,label='sum')
ax2.plot(time.value/1e6,FUV,label='sum')   
ax3.plot(time.value/1e6,Ha/FUV,label='sum')   
    
ax1.set(xlabel='time / Myr',ylabel='Halpha',xlim=[0,20])
ax2.set(xlabel='time / Myr',ylabel='FUV',xlim=[0,20])
ax3.set(xlabel='time / Myr',ylabel='Halpha / FUV',xlim=[0,20])

ax1.legend()
plt.tight_layout()
plt.show()

In [ ]:
def __add__(table1,table2):
    '''add two clusters with different ages
    
    this function takes the starburst
    '''
    
    time1 = table1['Time']
    
    time2 = table2['Time']
    
    HI_rate = np.interp(time1,time2,table2['HI_rate'],left=0,right=0)
    
    plt.plot(time1,table1['HI_rate'])
    plt.plot(time2,table2['HI_rate'])
    plt.plot(time1,table1['HI_rate']+HI_rate)
    plt.plot(time1,HI_rate)
    plt.show()
                                           
    return 0
    
__add__(cluster.quanta,c2.quanta)
    

In [ ]:
fig,(ax1,ax2,ax3) =plt.subplots(ncols=3,figsize=(two_column,two_column/3))


ax1.plot(cluster.ewidth['Time']/1e6,cluster.ewidth['Luminosity_H_A']/cluster.FUV['FUV'],color='black')
ax1.set(ylabel='Halpha/ FUV',xlabel='time / Myr',xlim=[0,10])

ax2.plot(cluster.ewidth['Time']/1e6,cluster.ewidth['Equ_width_H_A'],color='black')
ax2.set(ylabel='eq width / Angstrom',xlabel='time / Myr',xlim=[0,10])

ax3.plot(cluster.ewidth['Equ_width_H_A'],cluster.ewidth['Luminosity_H_A']/cluster.FUV['FUV'],color='black')
sc = ax3.scatter(cluster.ewidth['Equ_width_H_A'],cluster.ewidth['Luminosity_H_A']/cluster.FUV['FUV'],
                 c=cluster.ewidth['Time']/1e6,vmin=0,vmax=10)
fig.colorbar(sc,ax=ax3,label='age / Myr')
ax3.set(ylabel='Halpha/ FUV',xlabel='eq width')

plt.tight_layout()
plt.savefig(basedir/'reports'/'SB99_age_vs_Ha_over_FUV.png',dpi=600)
plt.show()

### number of ionizing photons

In [ ]:
catalogue.add_index('region_ID')
catalogue['Qpredicted'] = np.nan
for region_ID in catalogue['region_ID']:
    # the masses in the catalogue are off by a factor of 10
    mass = 0.1*catalogue['mass'][catalogue['region_ID']==region_ID]
    age  = catalogue['age'][catalogue['region_ID']==region_ID]*u.Myr
    scaled_cluster = cluster.scale(mass)
    idx = np.argmin(np.abs(scaled_cluster.quanta['Time']-age))
    catalogue.loc[region_ID]['Qpredicted'] = scaled_cluster.quanta['HI_rate'][idx].value

the conversion factor is from Niederhofer+2016

$$
Q(\mathrm{H}\alpha) = 7.31\cdot 10^{11} L(\mathrm{H}\alpha)
$$

In [ ]:
# observed
catalogue['L(Ha)'] = (catalogue['HA6562_flux']*1e-20*u.erg/u.s/u.cm**2 *4*np.pi*Distance(distmod=p['(m-M)'])**2).to(u.erg/u.s)
catalogue['Qobserved'] = 7.31e11*catalogue['L(Ha)']/u.erg
tmp = catalogue[(catalogue['mass']>1e3) & (catalogue['age']<6) & (catalogue['age']>0)]
# fesc = (Qpredicted-Qobserved) / Qpredicted 
fesc = (tmp['Qpredicted']-tmp['Qobserved'])/tmp['Qpredicted']

print(f"{np.sum(fesc<0)} of {len(tmp)} regions have negative fesc")
#Ha_from_q = (catalogue['Q']*1.37e-12*u.erg/u.s / (4*np.pi*Distance(distmod=p['(m-M)'])**2)).to(u.erg/u.s/u.cm**2)

In [ ]:
fig,ax =plt.subplots(figsize=(single_column,single_column))

Qpredicted = np.logspace(-2,2)

for f in [0.0,0.5,0.9,0.99]:
    Qobserved = Qpredicted*(1-f)
    ax.plot(Qpredicted,Qobserved,label=f'fesc={f}',zorder=1)

sc=ax.scatter(tmp['Qpredicted']/1e50,tmp['Qobserved']/1e50,
           c=tmp['age'],cmap=plt.cm.copper,vmin=0,vmax=6,s=2,zorder=2)
ax.legend()
fig.colorbar(sc,label='age / Myr')
#ax.plot([0,100],np.array([10,110]),color='gray',ls='--')
#ax.plot([0,100],np.array([-10,90]),color='gray',ls='--')

ax.set(xlabel=r'$Q_{\mathrm{H}\alpha}$ / $10^{50} \mathrm{s}^{-1}$ predicted',
       ylabel='$Q$ / $10^{50} \mathrm{s}^{-1}$ observed',
       xscale='log',yscale='log',xlim=[1e-2,1e2],ylim=[1e-2,1e2])
plt.savefig(basedir/'reports'/name/f'{name}_fesc.pdf',dpi=600)
plt.show()

In [ ]:
fig,ax =plt.subplots(figsize=(single_column,single_column))

ax.hist(fesc,bins=np.arange(0,1.1,0.05))
ax.set(xlim=[0,1.1],xlabel='fesc')
plt.show()

In [ ]:
fig,ax =plt.subplots(figsize=(single_column,single_column))

ax.scatter(tmp['galactic_radius'],fesc)
ax.set(ylim=[0,1.1])
plt.show()

### Filter response curve

to get the FUV flux by integrating the spectrum. The curves are from the [astrosat website](https://uvit.iiap.res.in/Instrument/Filters)



In [ ]:
from speclite.filters import FilterResponse, load_filters, plot_filters

response_curve = ascii.read(basedir/'data'/'external'/'astrosat_response_curve.txt',
                                     names=['wavelength','EA','Filter'])

F148W_mask = response_curve['Filter']=='F148W'
F148W_lam = response_curve['wavelength'][F148W_mask]*u.angstrom
F148W_res = response_curve['EA'][F148W_mask] / max(response_curve['EA'][F148W_mask])
F148W = FilterResponse(F148W_lam,F148W_res,meta=dict(group_name='Astrosat',band_name='F148W'))

F154W_mask = response_curve['Filter']=='F154W'
F154W_lam  = response_curve['wavelength'][F154W_mask]*u.angstrom
F154W_res  = response_curve['EA'][F154W_mask] / max(response_curve['EA'][F154W_mask])
F154W = FilterResponse(F154W_lam,F154W_res,meta=dict(group_name='Astrosat',band_name='F154W'))

astrosat_filter = load_filters('Astrosat-F148W', 'Astrosat-F154W')
plot_filters(astrosat_filter)

### Compare Halpha and FUV

In [ ]:
fig,ax1 =plt.subplots(figsize=(single_column,single_column))
Ha  = cluster.ewidth['Luminosity_H_A']
time_HA = cluster.ewidth['Time']

FUV  = cluster.FUV['FUV']
time_FUV = cluster.FUV['Time']

ax1.plot(time_HA/1e6,Ha,color='tab:red')
ax1.set_ylabel('Halpha/ (erg/s)',color='tab:red')
ax1.set(xlabel='Time/Myr')

ax2 = ax1.twinx() 
ax2.plot(time_FUV/1e6,FUV,color='tab:green')
ax2.set_ylabel('FUV / (erg/s)',color='tab:green')
ax2.set(xlabel='Time/Myr',xlim=[0,10])
plt.show()

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

FUV_int = np.interp(time_HA,time_FUV,FUV)

fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(two_column,two_column/2))
ax1.plot(time_HA/1e6,Ha/FUV_int,color='tab:blue')
ax1.set_ylabel('Halpha / FUV',color='tab:blue')
ax1.set(xlim=[0,3],ylim=[1e-5,0.0017],xlabel='Time/Myr')

axt = ax1.twinx()
quanta = cluster.quanta
axt.plot(quanta['Time']/1e6,quanta['HI_rate'],color='tab:orange')
axt.set_ylabel('ionizing photons / 1/s',color='tab:orange')
axt.set(xlim=[0,10])

HI_rate_int = np.interp(time_HA,quanta['Time'],quanta['HI_rate'])

ax2.plot(HI_rate_int,Ha/FUV_int,color='black')
sc = ax2.scatter(HI_rate_int,Ha/FUV_int,c=time_HA/1e6,vmin=0,vmax=10)
divider = make_axes_locatable(ax2)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(sc,cax=cax,label='age / Myr',pad=-1)

ax2.set(xlabel='ionizing photons / 1/s',ylabel='Halpha / FUV')

plt.tight_layout()
plt.show()

In [ ]:
fig,ax1 =plt.subplots(figsize=(single_column,single_column))

ax1.plot(time_HA/1e6,Ha/FUV_int,color='black')
#sc = ax1.scatter(FUV_int,Halpha,c=time_HA/1e6,vmin=0,vmax=3)
#fig.colorbar(sc,ax=ax1,label='age / Myr')
ax1.set(ylabel='Halpha/ FUV',xlabel='time / Myr',xlim=[0,10])
plt.savefig(basedir/'reports'/'age_vs_Ha_over_FUV.png',dpi=600)
plt.show()

In [ ]:
fig,ax1 =plt.subplots(figsize=(two_column,two_column/1.618))

for mass in [5e2,1e3,2e3,5e3,1e4,2e4,5e4]:
    
    scaled_cluster = cluster.scale(mass)
    
    Halpha  = scaled_cluster.ewidth['Luminosity_H_A']
    time_HA = scaled_cluster.ewidth['Time']
    FUV = scaled_cluster.FUV['FUV']
    
    ax1.plot(np.log10(FUV.value),np.log10(Halpha.value),color='black')
    ax1.text(np.log10(FUV.value)[0],np.log10(Halpha.value)[0],f'{mass:.0g}  ',
            horizontalalignment='right',verticalalignment='bottom')
    sc = ax1.scatter(np.log10(FUV.value),np.log10(Halpha.value),c=time_HA/1e6,vmin=0,vmax=30)

ax1.set(ylabel=r'log10 H$\alpha$ / (erg/s)',xlabel='log10 FUV / (erg/s)')
fig.colorbar(sc,label='age / Myr')
plt.show()

In [ ]:
clusters = {}
for m in [23,53,63,24,54,64]:
    clusters[m] = Cluster(stellar_model=m)

In [ ]:
fig,axes =plt.subplots(nrows=2,ncols=3,figsize=(two_column,two_column/1.618))
axes_iter = iter(axes.flatten())

for m in [23,53,63,24,54,64]:
    
    ax = next(axes_iter)
    cl = clusters[m]
    
    Halpha  = cl.ewidth['Luminosity_H_A']
    time_HA = cl.ewidth['Time']
    FUV = cl.FUV['FUV']
    
    label = f'{cl.stellar_model}, Z={cl.metallicity}'
    ax.plot(np.log10(FUV.value),np.log10(Halpha.value),color='black')
    #t = ax.text(0.05,0.9,label, transform=ax.transAxes,color='black',fontsize=8)
    ax.set_title(label,fontsize=8)
    sc = ax.scatter(np.log10(FUV.value),np.log10(Halpha.value),c=time_HA/1e6,vmin=0,vmax=30)

    ax.set(ylabel=r'H$\alpha$ / (erg/s)',xlabel='FUV / (erg/s)')

plt.tight_layout()

fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.9, 0.1, 0.05, 0.7])
fig.colorbar(sc, cax=cbar_ax,label='age / Myr')

plt.show()

In [ ]:
fig,(ax1,ax2) =plt.subplots(nrows=1,ncols=2,figsize=(two_column,two_column/2))

for m in [23,53,63]:
    
    cl = clusters[m]
    
    Halpha  = cl.ewidth['Luminosity_H_A']
    time_HA = cl.ewidth['Time']
    FUV = cl.FUV['FUV']
    
    label = f'{cl.stellar_model}, Z={cl.metallicity}'
    ax1.plot(np.log10(FUV.value),np.log10(Halpha.value),label=label)
    #t = ax.text(0.05,0.9,label, transform=ax.transAxes,color='black',fontsize=8)
    #sc = ax.scatter(np.log10(FUV.value),np.log10(Halpha.value),c=time_HA/1e6,vmin=0,vmax=30)

ax1.set(ylabel=r'H$\alpha$ / (erg/s)',xlabel='FUV / (erg/s)')
ax1.legend()

for m in [24,54,64]:
    
    cl = clusters[m]
    
    Halpha  = cl.ewidth['Luminosity_H_A']
    time_HA = cl.ewidth['Time']
    FUV = cl.FUV['FUV']

    label = f'{cl.stellar_model}, Z={cl.metallicity}'
    ax2.plot(np.log10(FUV.value),np.log10(Halpha.value),label=label)
    #t = ax.text(0.05,0.9,label, transform=ax.transAxes,color='black',fontsize=8)
    #sc = ax.scatter(np.log10(FUV.value),np.log10(Halpha.value),c=time_HA/1e6,vmin=0,vmax=30)

ax2.set(ylabel=r'H$\alpha$ / (erg/s)',xlabel='FUV / (erg/s)')
ax2.legend()
    
plt.tight_layout()


plt.show()

In [ ]:
fig,axes =plt.subplots(nrows=2,ncols=3,figsize=(two_column,two_column/1.618))
axes_iter = iter(axes.flatten())

for m in [23,53,63,24,54,64]:
    
    ax = next(axes_iter)
    cl = clusters[m]
    
    Halpha  = cl.ewidth['Luminosity_H_A']
    time_HA = cl.ewidth['Time']
    FUV = cl.FUV['FUV']
    ionizing = cl.quanta['HI_rate']
  
    label = f'{cl.stellar_model}, Z={cl.metallicity}'
    ax.plot(Halpha/FUV,ionizing,color='black')
    #t = ax.text(0.05,0.9,label, transform=ax.transAxes,color='black',fontsize=8)
    ax.set_title(label,fontsize=8)
    sc = ax.scatter(Halpha/FUV,ionizing,c=time_HA/1e6,vmin=0,vmax=10)

    ax.set(ylabel=r'H$\alpha$ / FUV',xlabel='ionization')

plt.tight_layout()

fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.9, 0.1, 0.05, 0.7])
fig.colorbar(sc, cax=cbar_ax,label='age / Myr')

plt.show()

### Compare observations to simulations

In [ ]:
catalogue['distance'] = np.nan
for name in np.unique(catalogue['gal_name']):
    catalogue['distance'][catalogue['gal_name']==name] = (sample_table.loc[name]['Distance']*u.Mpc).to(u.cm)

In [ ]:
fig,ax1 =plt.subplots(figsize=(two_column,two_column/1.618))

for mass in [2e3,5e3,1e4,2e4,5e4,1e5,2e5,5e5,1e6,2e6,5e6]:
    
    scaled_cluster = cluster.scale(mass)
    
    Halpha  = scaled_cluster.ewidth['Luminosity_H_A']
    time_HA = scaled_cluster.ewidth['Time']
    FUV = scaled_cluster.FUV['FUV']
    
    ax1.plot(np.log10(FUV.value),np.log10(Halpha.value),color='black')
    ax1.text(np.log10(FUV.value)[0],np.log10(Halpha.value)[0],f'{mass:.0g}  ',
            horizontalalignment='right',verticalalignment='bottom')
    sc = ax1.scatter(np.log10(FUV.value),np.log10(Halpha.value),c=time_HA/1e6,vmin=0,vmax=30)

Halpha_FLUX = ((catalogue['HA6562_FLUX_CORR']*1e-20*u.erg/u.s/u.cm**2) * 4*np.pi*catalogue['distance']**2).value
FUV_FLUX = 5e5*((catalogue['FUV_FLUX_CORR']*1e-20*u.erg/u.s/u.cm**2) * 4*np.pi*catalogue['distance']**2).value
ax1.scatter(np.log10(FUV_FLUX),np.log10(Halpha_FLUX))

ax1.set(ylabel=r'log10 H$\alpha$ / (erg/s)',xlabel='log10 FUV / (erg/s)')
fig.colorbar(sc,label='age / Myr')
plt.show()

create the grid to compare the observations to

In [ ]:
n_time = len(cluster.ewidth['Time'])
n_mass = 1000

mass_min = 5e3
mass_max = 1e6

mass_grid = np.linspace(mass_min,mass_max,n_mass)

HA_grid = np.zeros((n_time,n_mass))
FUV_grid = np.zeros((n_time,n_mass))

for i,mass in enumerate(mass_grid):
    
    scaled_cluster = cluster.scale(mass)
    
    HA_grid[:,i]  = scaled_cluster.ewidth['Luminosity_H_A']
    FUV_grid[:,i] = scaled_cluster.FUV['FUV']

time = scaled_cluster.FUV['Time']

In [ ]:
#distance = (sample_table.loc[name]['Distance']*u.Mpc).to(u.cm)

mass, age, chi2 = [],[],[]
for row in catalogue:
    
    Halpha_FLUX = ((row['HA6562_FLUX_CORR']*1e-20*u.erg/u.s/u.cm**2) * 4*np.pi*row['distance']**2).value
    Halpha_ERR  = ((row['HA6562_FLUX_CORR_ERR']*1e-20*u.erg/u.s/u.cm**2) * 4*np.pi*row['distance']**2).value
    
    FUV_FLUX = 1e6*((row['FUV_FLUX_CORR']*1e-20*u.erg/u.s/u.cm**2) * 4*np.pi*row['distance']**2).value
    FUV_ERR  = ((row['FUV_FLUX_CORR_ERR']*1e-20*u.erg/u.s/u.cm**2) * 4*np.pi*row['distance']**2).value
    
    chi2_grid = (Halpha_FLUX-HA_grid)**2/Halpha_ERR**2 + (FUV_FLUX-FUV_grid)**2/FUV_ERR**2
    
    row,col = np.unravel_index(chi2_grid.argmin(), chi2_grid.shape)
    mass.append(mass_grid[col])
    age.append(time[row].value)
    chi2.append(np.min(chi2_grid))
    

In [ ]:
fig,(ax1,ax2) =plt.subplots(nrows=1,ncols=2,figsize=(two_column,two_column/2))

ax1.scatter(np.array(age)/1e6,catalogue['AGE_MINCHISQ'])
ax1.set(ylim=[0,30],xlim=[0,30],xlabel='age from Nebulae / Myr',ylabel='age from Cluster / Myr')

ax2.scatter(np.array(mass),catalogue['MASS_MINCHISQ'])
ax2.set(xlabel='mass from Nebulae / Msun',ylabel='mass from Cluster / Msun')

plt.tight_layout()
plt.show()

In [ ]:
fig,ax1 =plt.subplots(nrows=1,ncols=1,figsize=(single_column,single_column))

ax1.scatter(np.array(age)/1e6,np.array(mass))
ax1.set(xlim=[0,30],ylim=[0,5e5],xlabel='age from Nebulae / Myr',ylabel='mass / Msun')

plt.tight_layout()
plt.show()

In [ ]:
fig,ax1 =plt.subplots(figsize=(single_column,single_column))
eqwHA  = cluster.ewidth['Equ_width_H_A']
time_HA = cluster.ewidth['Time']

ax1.plot(time_HA/1e6,eqwHA,color='tab:red')
ax1.set(xlabel='Time/Myr',ylabel='eq / AA',xlim=[0,10])

#ax2 = ax1.twinx() 
#ax2.plot(time_FUV/1e6,FUV,color='tab:green')
#ax2.set_ylabel('FUV / (erg/s)',color='tab:green')
#ax2.set(xlabel='Time/Myr',xlim=[0,10])
plt.show()

In [ ]:
fig,ax1 =plt.subplots(figsize=(single_column,single_column))
colors = ['tab:blue','tab:cyan','tab:red','tab:orange','tab:green','tab:olive']

for m,c in zip([23,24,53,54,63,64],colors):
    
    cl = clusters[m]
    
    eqwHA  = cl.ewidth['Equ_width_H_A']
    time_HA = cl.ewidth['Time']
    ax1.plot(np.log10(time_HA/u.yr),np.log10(eqwHA/u.angstrom),label=f'{cl.stellar_model }',color=c)
    
    #sl = cl.scale(1e5)
    #eqwHA  = sl.ewidth['Equ_width_H_A']
    #time_HA = sl.ewidth['Time']
    #ax1.plot(np.log10(time_HA/u.yr),np.log10(eqwHA/u.angstrom),ls='--',color=c)
    
    
ax1.set(xlabel='log (Time / Myr)',ylabel=r'log (W(H$\alpha$) / $\AA$)',xlim=[6,7.5])
ax1.legend()
#plt.savefig(basedir/'reports'/'equivalent_width_vs_age.pdf',dpi=600)
plt.show()

In [ ]:
fig,ax1 =plt.subplots(figsize=(single_column,single_column))
colors = ['tab:blue','tab:cyan','tab:red','tab:orange','tab:green','tab:olive']

for m,c in zip([23,24,53,54,63,64],colors):
    
    cl = clusters[m]
    
    HAFUV  = cl.ewidth['Luminosity_H_A'] / cl.FUV['FUV']
    time_HA = cl.ewidth['Time']
    ax1.plot(time_HA/1e6,HAFUV,label=f'{cl.stellar_model }',color=c)
    
    sl = cl.scale(1e5)
    HAFUV  = sl.ewidth['Luminosity_H_A'] / sl.FUV['FUV']
    time_HA = sl.ewidth['Time']
    ax1.plot(time_HA/1e6,HAFUV,ls='--',color=c)    
    
ax1.set(xlabel='Time / Myr',ylabel=r'HA/FUV',xlim=[0,10])
ax1.legend()
#plt.savefig(basedir/'reports'/'equivalent_width_vs_age.pdf',dpi=600)
plt.show()

### Create custom cluster

In [ ]:
def create_cluster(folder,**kwargs):

    parameters = {
    "name" : 'standard',
    "isf" : -1,
    "mass" : 1.,
    "sfr" : 1,
    "ninterv" : 2,
    "xponent" : '1.3,2.3',
    "xmaslim" : '0.1,0.5,120',
    "sncut" : 8.,
    "bhcut" : 120.,
    "model" : 64,
    "wind_model" : 0,
    "tinitial" : 0.01,
    "time_scale" : 0,
    "time_step" : 0.1,
    "n_steps" : 1000,
    "tmax" : 50,
    "jmg" : 3,
    "atmos" : 5,
    "metallicity" : 3,
    "uvline" : 1}

    # assign the new parameters
    for k,v in kwargs.items():
        if k in parameters:
            parameters[k] = v
    
    # open tempalte
    with open(basedir/'data'/'input.template') as f:
        template = f.read()
    
    template = template.format(**parameters)

    #write to templae
    with open(folder/'input.out','w') as f:
        f.write(template)
        
        


## Playground

### Principal component analysis

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
columns = ['age','mass','HA/FUV','HA6562_FLUX','region_area']


data = np.zeros((len(catalogue),len(columns)))

for i,col in enumerate(columns):
    data[:,i] = catalogue[col].data
data = StandardScaler().fit_transform(data)

In [ ]:
pca_nebulae = PCA(n_components=2)
principalComponents = pca_nebulae.fit_transform(data)

### Sitelle

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(ncols=3,figsize=(10,4))

for name,ax in zip(['NGC0628','NGC2835','NGC3351'],[ax1,ax2,ax3]):
    with fits.open(basedir/'..'/'sitelle'/'data'/'maps'/f'{name}_OII_map.fits') as hdul:
        OII = hdul['OII3726_FLUX'].data
        OII_header = hdul['OII3726_FLUX'].header
    
    norm = simple_norm(OII,clip=False,percent=98)
    ax.imshow(OII,norm=norm,origin='lower',cmap=plt.cm.Greys)
    ax.set_title(name)
    ax.axis('off')
    
plt.show()   

use [OII] line to calculate strong line and direct abundances

In [ ]:
from cluster.metallicity import diagnostic_line_ratios

with fits.open(basedir/'..'/'sitelle'/'data'/'Nebulae_Catalogue_DR2_native_with_OII.fits') as hdul:
    nebulae = Table(hdul[1].data)

nebulae = diagnostic_line_ratios(nebulae)

compare with Figure 8 in Pilyugin+2016 (looks good if R2*=1.4)

In [ ]:
from cluster.metallicity import strong_line_metallicity_R, strong_line_metallicity_S

subsample = nebulae[nebulae['OII3726_FLUX_CORR']>10*nebulae['OII3726_FLUX_CORR_ERR']].copy()
subsample['OH_R'] = strong_line_metallicity_R(1.4*subsample['R2'],subsample['R3'],subsample['N2'])
subsample['OH_S'] = strong_line_metallicity_S(subsample['S2'],subsample['R3'],subsample['N2'])

fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(6,3))

ax1.plot([8.1,8.8],[8.1,8.8],color='black')
ax1.plot([8.1,8.8],[8.2,8.9],color='grey',ls='--')
ax1.plot([8.1,8.8],[8.0,8.7],color='grey',ls='--')
ax1.scatter(subsample['OH_S'],subsample['OH_R'],s=4,c=tab10[0])
ax1.set(xlim=[8.1,8.8],ylim=[8.1,8.8],
       xlabel='12+log(O/H)$_\mathrm{S}$',
       ylabel='12+log(O/H)$_\mathrm{R}$')
ax2.hist(subsample['OH_S']-subsample['OH_R'],bins=np.linspace(-0.3,0.3,20),histtype='step',color='black')
ax2.set(xlabel=r'log(O/H)$_\mathrm{S}-$log(O/H)$_\mathrm{R}$')
plt.savefig('12+logOH R vs S calibration.png',dpi=600)
plt.show()

with direct method. This requires electron temperature and density. They have to be measured in an itterative process

In [ ]:
from cluster.metallicity import electron_density_sulfur,\
                                electron_temperature_oxygen, electron_temperature_nitrogen,\
                                electron_temperature_sulfur, oxygen_abundance_direct
   
criteria = (nebulae['OII7319_FLUX_CORR']>7*nebulae['OII7319_FLUX_CORR_ERR']) & (nebulae['OII3726_FLUX_CORR']>10*nebulae['OII3726_FLUX_CORR_ERR'])
subsample = nebulae[criteria].copy()
subsample['OH_R'] = strong_line_metallicity_R(subsample['R2'],subsample['R3'],subsample['N2'])
subsample['OH_S'] = strong_line_metallicity_S(subsample['S2'],subsample['R3'],subsample['N2'])
    
# initial guess for the temperature
subsample['t(NII)'] = electron_temperature_nitrogen(subsample['RN2'])
subsample['t(SIII)'] = electron_temperature_sulfur(subsample['RS3'])
subsample['n(SII)']  = electron_density_sulfur(subsample['RS2'],subsample['t(NII)'])

for x in range(10):
    subsample['t(OII)'] = electron_temperature_oxygen(subsample['RO2'],subsample['n(SII)'])
    subsample['n(SII)'] = electron_density_sulfur(subsample['RS2'],subsample['t(OII)'])
    print(np.nanmean(subsample['n(SII)']))

subsample['OH_direct'] = oxygen_abundance_direct(subsample['R2'],subsample['R3'],subsample['t(OII)'],subsample['n(SII)'])


In [ ]:
fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(6,3))

ax1.plot([8.1,8.8],[8.1,8.8],color='black')
ax1.plot([8.1,8.8],[8.2,8.9],color='grey',ls='--')
ax1.plot([8.1,8.8],[8.0,8.7],color='grey',ls='--')
ax1.scatter(subsample['OH_direct'],subsample['OH_R'],s=4,c=tab10[0])
ax1.set(xlim=[8.1,8.8],ylim=[8.1,8.8],
       xlabel='12+log(O/H) direct',
       ylabel='12+log(O/H)$_\mathrm{R}$')
ax2.hist(subsample['OH_direct']-subsample['OH_R'],bins=np.linspace(-0.3,0.3,20),histtype='step',color='black')
ax2.set(xlabel=r'log(O/H) direct$-$log(O/H)$_\mathrm{R}$')
#plt.savefig('12+logOH R vs S calibration.png',dpi=600)
plt.show()

compare with Figure 7 in Perez-Montero+2017

In [ ]:
fig,ax=plt.subplots(figsize=(6,4))

ax.scatter(np.log10(subsample['R23']),subsample['OH_direct'],s=4,c=subsample['logq_D91'])
ax.set(xlim=[-0.4,1.4],ylim=[7.,9.0],
       xlabel='log R23',
       ylabel='12+log(O/H) direct')
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(4,4))

ax.plot([0.5,1.5],[0.5,1.5],color='black')
ax.scatter(subsample['t(OII)'],subsample['t(NII)'],s=4,c=tab10[0])
ax.set(xlim=[0.5,1.5],ylim=[0.5,1.5],
       xlabel='t([OII])',
       ylabel='t([NII])')
#plt.savefig('12+logOH R vs S calibration.png',dpi=600)
plt.show()

### PyNeb

In [ ]:
import pyneb as pn

In [ ]:
O3 = pn.Atom('O',3, NLevels=5)
O2 = pn.Atom('O',2, NLevels=5)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(13,7))
O2.plotGrotrian(ax=ax1)
O3.plotGrotrian(ax=ax2)

In [ ]:
obs = pn.Observation()

for line,wave in zip(['OI6300','OII3726','OII7319','OII7330','OIII5006','NII5754',
             'NII6583','SII6716','SII6730','SIII6312','SIII9068'],
               [6300,3726,7319,7330,5007,5755,6584,6716,6731,6312,9069], 
               ):
    Intens = subsample[f'{line}_FLUX_CORR']
    Error  = subsample[f'{line}_FLUX_CORR_ERR']
    line = pn.EmissionLine(line[0],len(line[1:-4]),wave,obsIntens=Intens,obsError=Error)
    
    obs.addLine(line)
    
diags = pn.Diagnostics()
diags.addDiagsFromObs(obs)

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
emisgrids = pn.getEmisGridDict(atomDict=diags.atomDict)
diags.plot(emisgrids, obs, ax=ax,i_obs=30)

In [ ]:
te,ne = diags.getCrossTemDen(diag_tem='[NII] 5755/6584',diag_den='[SII] 6731/6716',obs=obs)

In [ ]:
Te = [10000.]
Ne = [1e3]
# Define a dictionary to hold all the Atom objects needed
all_atoms = pn.getAtomDict(atom_list=obs.getUniqueAtoms())
# define a dictionary to store the abundances
ab_dict = {}
# we  use the following lines to determine the ionic abundances
ab_labels = ['O3_5007A', 'H1r_4861A', 'O2_3726A', 'O2_7319A', 'O2_7330A',
             'N2_5755A', 'N2_6584A', 'S2_6716A', 'S2_6731A', 'S3_6312A','S3_9069A']

for line in obs.getSortedLines():
    if line.label in ab_labels:
        ab = all_atoms[line.atom].getIonAbundance(line.corrIntens, Te, Ne, 
                                                  to_eval=line.to_eval, Hbeta=100)
        ab_dict[line.atom] = ab



In [ ]:
from astropy.coordinates import match_coordinates_sky
idx,sep,_=match_coordinates_sky(associations['SkyCoord'],nebulae['SkyCoord'])

sep = sep.to(u.arcsec).value

fig,ax=plt.subplots()
bins = np.arange(0,10)

print(f"x<0.5: {np.mean(associations[sep<0.5]['age']):.2f}")
print(f"0.5<x<1: {np.mean(associations[(sep>0.5) & (sep<1)]['age']):.2f}")
print(f"1<x: {np.mean(associations[sep>1]['age']):.2f}")

ax.hist(associations[sep<0.5]['age'],bins=bins,alpha=0.5,label=r'$x<0.5"$')
ax.hist(associations[(sep>0.5) & (sep<1)]['age'],bins=bins,alpha=0.5,label=r'$0.5"<x<1"$')
ax.hist(associations[sep>1]['age'],bins=bins,alpha=0.5,label=r'$1"<x$')

ax.legend()
ax.set(xlim=[0,10])
plt.show()